In [1]:
from __future__ import print_function
import IPython
import sys
import numpy as np
import os
import re
import pandas as pd
from tqdm import tqdm
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import librosa as sf
import librosa.display as sfd
%matplotlib inline

/Users/dandamian/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
nf = './zgomot-fond.m4a'
sample = './sample-cry/'

In [9]:
from pydub import AudioSegment
ios_paths = [os.path.join(sample, x) for x in os.listdir(sample)]

combined = AudioSegment.from_file(nf)
pos = 0
delta = 60000
for p in tqdm(ios_paths):
    pos = pos + delta
    cry = AudioSegment.from_file(p)
    combined = combined.overlay(cry, position=pos)

combined.export("mix.wav", format='wav')

100%|██████████| 30/30 [00:18<00:00,  1.61it/s]


<_io.BufferedRandom name='mix.wav'>

In [41]:
audio, sr = sf.load('mix.wav', sr=16000)

In [42]:
sound_len = len(audio)/sr / 60
print(sound_len)

28.325316666666666


In [43]:
audio = np.array(audio)

In [44]:
27192304 / 16000

1699.519

In [45]:
1699*16000

27184000

In [46]:
audio_cut = audio[0:27184000]

In [47]:
audio_set = audio_cut.reshape(1699, 16000)

In [51]:
audio_set.shape

(1699, 16000)

In [58]:
IPython.display.Audio(audio_set[181],rate=16000)

In [59]:
target = np.zeros(1699)

In [60]:
for i in range(1699):
    if i % 60 == 0:
        for j in range (7):
            target[i+j] = 1

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(audio_set, target, test_size=0.33, random_state=42)

In [71]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=X_train[0].shape))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [72]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                1024064   
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 1,028,289
Trainable params: 1,028,289
Non-trainable params: 0
_________________________________________________________________


In [75]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [78]:
model.fit(X_train, Y_train, epochs=30, batch_size=16)

Epoch 1/30
1138/1138 [==============================] - 1s 1ms/step - loss: 0.0062 - acc: 0.9982
Epoch 2/30
1138/1138 [==============================] - 2s 1ms/step - loss: 0.0163 - acc: 0.9982
Epoch 3/30
1138/1138 [==============================] - 1s 1ms/step - loss: 0.0068 - acc: 0.9991
Epoch 4/30
1138/1138 [==============================] - 1s 1ms/step - loss: 0.0016 - acc: 1.0000
Epoch 5/30
1138/1138 [==============================] - 1s 1ms/step - loss: 0.0012 - acc: 1.0000
Epoch 6/30
1138/1138 [==============================] - 1s 1ms/step - loss: 0.0026 - acc: 0.9991
Epoch 7/30
1138/1138 [==============================] - 2s 1ms/step - loss: 0.0011 - acc: 1.0000
Epoch 8/30
1138/1138 [==============================] - 1s 1ms/step - loss: 0.0029 - acc: 0.9982
Epoch 9/30
1138/1138 [==============================] - 1s 1ms/step - loss: 0.0156 - acc: 0.9991
Epoch 10/30
1138/1138 [==============================] - 1s 1ms/step - loss: 0.0013 - acc: 1.0000
Epoch 11/30
1138/1138 [======

In [79]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 1.5186039700227625
Test accuracy: 0.8752228166117813
